In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import networkx as nx
import patsy
import pandas as pd
import scipy.stats as stats

# put TAM_main into this directory
from TAM_main.utils import *  

We first define several functions.

In [ ]:
## function: generate sufficient stat


def gen_suff_stat(X, basis_setting):
    basis_type = basis_setting['type']
    
    if basis_type == 'spline':
        n_knots = basis_setting['n_knots']
        space = 1 / (n_knots + 1)
        Q = space * np.arange(1, n_knots + 1)
        degree = basis_setting['degree']
        include_intercept = basis_setting['include_intercept']
        def gen_basis(x):
            return patsy.bs(x, knots = np.quantile(x, Q), degree = degree, include_intercept = include_intercept)
    
    if basis_type == 'rbf_k':
        nb = basis_setting['nb']
        gamma = basis_setting['gamma']
        def gen_basis(x):
            return rbf_kernel(x, nb = nb, gamma = gamma)
    
    Xsp = {}
    n, p = X.shape
    for j in range(p):
        A = gen_basis(X[:,j])
        A = A - np.mean(A, axis = 0)
        Xsp[j] = A
    
    R = A.shape[1]
    
    Z = X
    for r in range(R):
        for j in range(p):
            Z = np.append(Z, Xsp[j][:,r].reshape(-1,1), axis = 1)
            
    Sigma = Z.T @ Z / n
    
    return {"Sigma": Sigma, "R": R}

In [ ]:
## function: MIP for equal variance


def Mips_DAG_equalvar(Sigma_hat, n, p, lam, R, moral, time_limit = 300, known_edges = [], partial_order = [], start_edges = [], tau = 0.001, verbose = 1):
    
    # find the superset
    E = [(i, j) for i in range(p) for j in range(p) if i != j]  # off diagonal edge sets
    list_edges = []
    for edge in moral:
        list_edges.append((edge[0], edge[1]))
        list_edges.append((edge[1], edge[0]))
    G_moral = nx.Graph()
    for i in range(p):
        G_moral.add_node(i)
        G_moral.add_edges_from(list_edges)
    non_edges = list(set(E) - set(list_edges))
    
    # ! model setup !
    m = gp.Model()

    # ! Create variables !
    # Continuous variables
    # set Gamma (which is essentially Beta)
    Gamma = {}
    for i in range(p):
        for j in range(p):
            if i == j:
                Gamma[i,j] = 1
            else:
                Gamma[i,j] = 0
    for r in range(1,(R+1)):
        for j in range(p):
            for i in range(p):
                if (i,j) in list_edges:
                    Gamma[j,i+p*r] = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Gamma_%s_%s_%s" % (i, j, r))
                else:
                    Gamma[j,i+p*r] = 0
    
    # set psi
    psi = m.addMVar((p, 1), lb=1, ub=p, vtype=GRB.CONTINUOUS, name='psi')
    
    # Integer variables
    g = {}
    for i,j in list_edges:
        if (i,j) in known_edges:
            g[i,j] = 1
        else:
            g[i,j] = m.addVar(vtype=GRB.BINARY, name="g_%s_%s" % (i, j))
            
    # ! Set objective !
    # trace term
    trace = gp.QuadExpr()
    for k in range((R+1)*p):
        for i in range((R+1)*p):
            for j in range(p):
                trace += Gamma[j,k] * Gamma[j, i] * Sigma_hat[i, k]
    
    # penalty term
    penalty = gp.LinExpr()
    for i,j in list_edges:
        penalty += lam * g[i, j]
        
    # ! Solve the problem without constraints to get big_M !
    m.setObjective(trace + penalty, GRB.MINIMIZE)
    m.Params.OutputFlag = verbose
    m.Params.TimeLimit = time_limit
    m.update()
    m.optimize()
    
    big_M = 0
    for r in range(1,(R+1)):
        for i,j in list_edges:
            big_M = max(big_M, abs(Gamma[j,i+p*r].x))
    M = 2*big_M
    
    # ! Set constraints !    
    for r in range(1,(R+1)):
        for i,j in list_edges:
            m.addConstr(M*g[i,j] >= Gamma[j,i+p*r])
            m.addConstr(-M*g[i,j] <= Gamma[j,i+p*r])
    
    for i,j in list_edges:
        m.addConstr(1-p+p*g[i, j] <= psi[j] - psi[i])
    
    for i,j in partial_order:
        m.addConstr(psi[i] <= psi[j])

        
    # ! solve the problem !
    m.setObjective(trace + penalty, GRB.MINIMIZE)
    # m.Params.TimeLimit = timelimit*p
    m.Params.OutputFlag = verbose
    m.Params.TimeLimit = time_limit
    if tau > 0:
        m.Params.MIPGapAbs = tau
    m.update()
    m.optimize()
    
    # ! get vars !
    Beta_val = np.zeros((p, p * (R+1)))
    for var in m.getVars():
        if "Gamma" in var.varName:
            name = var.varName.split("_")[1:]
            i = int(name[0])
            j = int(name[1])
            r = int(name[2])
            Beta_val[j,i+p*r] = var.X
    for i in range(p):
        Beta_val[i,i] = 1
    
    g_val = []
    for var in m.getVars():
        if "g" in var.varName and var.X == 1:
            name = var.varName.split("_")[1:]
            g_val.append((int(name[0]), int(name[1])))
    for i,j in list_edges:
        if (i,j) in known_edges:
            g_val.append((i,j))
            
    psi_val = {}
    for var in m.getVars():
        if "psi" in var.varName:
            name = var.varName.split('[')[1].split(']')[0].split(',')
            psi_val[int(name[0])] = var.X
            
    # how to estimate variance?
    varhat = np.sum(np.diag(Beta_val @ Sigma_hat @ Beta_val.T)) / p
    Gamma_val = Beta_val / np.sqrt(varhat)

    
    return {"Beta": Beta_val, "Gamma": Gamma_val, "g":g_val, "psi":psi_val, "var": varhat}

In [ ]:
## function: MIP for unequal variance


def Mips_DAG_diffvar(Sigma_hat, n, p, lam, R, moral, time_limit = 300, known_edges = [], partial_order = [], start_edges = [], tau = 0.001, verbose = 1):
    
    # find the superset
    E = [(i, j) for i in range(p) for j in range(p) if i != j]  # off diagonal edge sets
    list_edges = []
    for edge in moral:
        list_edges.append((edge[0], edge[1]))
        list_edges.append((edge[1], edge[0]))
    G_moral = nx.Graph()
    for i in range(p):
        G_moral.add_node(i)
        G_moral.add_edges_from(list_edges)
    non_edges = list(set(E) - set(list_edges))
    
    # ! model setup !
    m = gp.Model()
    
    # ! Create variables !
    # Continuous variables
    # set Gamma
    Gamma = {}
    for i in range(p):
        for j in range(p):
            if i == j:
                Gamma[i,j] = m.addVar(lb=1e-5, vtype=GRB.CONTINUOUS, name="Gamma_%s_%s_%s" % (i, j, 0))
            else:
                Gamma[i,j] = 0
    for r in range(1,(R+1)):
        for j in range(p):
            for i in range(p):
                if (i,j) in list_edges:
                    Gamma[j,i+p*r] = m.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Gamma_%s_%s_%s" % (i, j, r))
                else:
                    Gamma[j,i+p*r] = 0
    
    # set psi
    psi = m.addMVar((p, 1), lb=1, ub=p, vtype=GRB.CONTINUOUS, name='psi')
    
    # Integer variables
    g = {}
    for i,j in list_edges:
        if (i,j) in known_edges:
            g[i,j] = 1
        else:
            g[i,j] = m.addVar(vtype=GRB.BINARY, name="g_%s_%s" % (i, j))
        
    # Variables for outer approximation
    T = {}
    for i in range(p):
        T[i] = m.addVar(lb=-10, ub=100, vtype=GRB.CONTINUOUS, name="T_%s" % i)
        # This gives Gamma[i,i] a range about [0.0001, 100]
            
    for i in range(p):
        m.addGenConstrLog(Gamma[i,i], T[i])
    
    # ! Set objective !
    # log term
    log_term = gp.LinExpr()
    for i in range(p):
        log_term += -2 * T[i]
    
    # trace term
    trace = gp.QuadExpr()
    for k in range((R+1)*p):
        for i in range((R+1)*p):
            for j in range(p):
                trace += Gamma[j,k] * Gamma[j, i] * Sigma_hat[i, k]
    
    # penalty term
    penalty = gp.LinExpr()
    for i,j in list_edges:
        penalty += lam * g[i, j]
        
    # ! Solve the problem without constraints to get big_M !
    m.setObjective(log_term + trace + penalty, GRB.MINIMIZE)
    m.Params.OutputFlag = verbose
    m.Params.TimeLimit = time_limit
    m.update()
    m.optimize()
    
    big_M = 0
    for i in range(p):
        big_M = max(big_M, abs(Gamma[i, i].x))
    for r in range(1,(R+1)):
        for i,j in list_edges:
            big_M = max(big_M, abs(Gamma[j,i+p*r].x))
    M = 2*big_M
    
    # ! Set constraints !    
    m.addConstrs(Gamma[i, i] <= M for i in range(p))
    for i in range(p):
        m.addGenConstrLog(Gamma[i,i], T[i])
    
    for r in range(1,(R+1)):
        for i,j in list_edges:
            m.addConstr(M*g[i,j] >= Gamma[j,i+p*r])
            m.addConstr(-M*g[i,j] <= Gamma[j,i+p*r])
    
    for i,j in list_edges:
        m.addConstr(1-p+p*g[i, j] <= psi[j] - psi[i])
    
    for i,j in partial_order:
        m.addConstr(psi[i] <= psi[j])

        
    # ! solve the problem !
    m.setObjective(log_term + trace + penalty, GRB.MINIMIZE)
    # m.Params.TimeLimit = timelimit*p
    m.Params.OutputFlag = verbose
    m.Params.TimeLimit = time_limit
    if tau > 0:
        m.Params.MIPGapAbs = tau
    m.update()
    m.optimize()

    # ! get vars !
    Gamma_val = np.zeros((p, p * (R+1)))
    for var in m.getVars():
        if "Gamma" in var.varName:
            name = var.varName.split("_")[1:]
            i = int(name[0])
            j = int(name[1])
            r = int(name[2])
            Gamma_val[j,i+p*r] = var.X
            
    g_val = []
    for var in m.getVars():
        if "g" in var.varName and var.X == 1:
            name = var.varName.split("_")[1:]
            g_val.append((int(name[0]), int(name[1])))
    for i,j in list_edges:
        if (i,j) in known_edges:
            g_val.append((i,j))
            
    psi_val = {}
    for var in m.getVars():
        if "psi" in var.varName:
            name = var.varName.split('[')[1].split(']')[0].split(',')
            psi_val[int(name[0])] = var.X
    
    # how to estimate variance?
    varhat = 1 / ( np.diag(Gamma_val) ** 2 )
    Beta_val = Gamma_val * np.sqrt(varhat)[:, np.newaxis]
    
    
    return {"Beta": Beta_val, "Gamma": Gamma_val, "g":g_val, "psi":psi_val, "varhat": varhat}

In [ ]:
## function: transform edges to adjacent matrix


def edge2adj(edges, p):
    adj = np.zeros((p,p))
    for (i,j) in edges:
        adj[i,j] = 1
    return adj

In [ ]:
## function: compute loss function from estimated edges


def loss_from_edges(edges, W, basis_setting):
    n, p = W.shape
    basis_type = basis_setting['type']
    
    if basis_type == 'spline':
        n_knots = basis_setting['n_knots']
        space = 1 / (n_knots + 1)
        Q = space * np.arange(1, n_knots + 1)
        degree = basis_setting['degree']
        include_intercept = basis_setting['include_intercept']
        def gen_basis(x):
            return patsy.bs(x, knots = np.quantile(x, Q), degree = degree, include_intercept = include_intercept)
    
    else: 
        def gen_basis(x):
            return x.reshape(-1, 1)

    
    # find adjacency matrix
    Adj_mat = edge2adj(edges, p)
    var_hat = np.array([])
    
    # find variances
    for i in range(p):
        idx = np.nonzero(Adj_mat[:,i])[0]
        if(idx.size != 0):
            y = W[:,i]
            X = W[:,idx]
            # find basis functions
            p_ = X.shape[1]
            Z = np.zeros((n, 0))
            for j in range(p_):
                A = gen_basis(X[:,j])
                A = A - np.mean(A, axis = 0)
                Z = np.append(Z, A, axis = 1)
            r = y - Z @ np.linalg.inv(Z.T @ Z) @ Z.T @ y
        else:
            r = W[:,i]
        
        var_hat = np.append(var_hat, np.square(np.linalg.norm(r, ord = 2)) / n )
    
    # find loss value
    loss_val = np.sum(np.log(var_hat)) + p
    return(loss_val)

Now we implement MIP nonlinear and MIP linear.

We repeat the following for each function h(x). 

We take $h(x) = \exp(x)$ as an example. That means, we read data $X$ that is associated with $h(x) = \exp(x)$.

In [ ]:
edges_star = edge2adj([(0,1), (0,2), (1,2)], 5)
RS = np.zeros((0, 5))

In [ ]:
for n in range(100, 2001, 100):
    for m in range(1, 101, 1):

        # for equal ======================================================================
        name = "./data/equal_" + str(n) + "_" + str(m) + ".csv"
        X = np.loadtxt(name, delimiter=" ")
        n,p = X.shape
        basis_setting = {'type': 'spline', 'n_knots': 5, 'degree': 3, 'include_intercept': False}
        suff = gen_suff_stat(X, basis_setting)
        Sigma_hat = suff["Sigma"]
        R = suff["R"]
        lam = 0.01
        moral = [(i,j) for i in range(5) for j in range(5)]

        # mip
        mips_equalvar_obj = Mips_DAG_equalvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs1 = loss_from_edges(mips_equalvar_obj['g'], X, basis_setting) + len(mips_equalvar_obj['g']) * np.log(n) / n
        mips_diffvar_obj = Mips_DAG_diffvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs2 = loss_from_edges(mips_diffvar_obj['g'], X, basis_setting) + len(mips_diffvar_obj['g']) * np.log(n) / n + (p) * np.log(n) / n
        if rs1 < rs2:
            RS = np.vstack([
                RS, 
                np.array([n, m, 'equal', 'Mips', 
                    np.all(edge2adj(mips_equalvar_obj['g'], p) == edges_star),
                    np.sum(np.abs( edge2adj(mips_equalvar_obj['g'], p) - edges_star )) 
                ])
            ])
        else:
            RS = np.vstack([
                RS, 
                np.array([n, m, 'equal', 'Mips', 
                    np.all(edge2adj(mips_diffvar_obj['g'], p) == edges_star ),
                    np.sum(np.abs( edge2adj(mips_diffvar_obj['g'], p) - edges_star )) 
                ])
            ])


        # linear
        Z = np.append(X, X, axis = 1)
        Sigma_hat = Z.T @ Z / n
        R = 1
        linear_equalvar_obj = Mips_DAG_equalvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs1 = loss_from_edges(linear_equalvar_obj['g'], X, basis_setting = {'type': 'linear'}) + len(linear_equalvar_obj['g']) * np.log(n) / n
        linear_diffvar_obj = Mips_DAG_diffvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs2 = loss_from_edges(linear_diffvar_obj['g'], X, basis_setting = {'type': 'linear'}) + len(linear_diffvar_obj['g']) * np.log(n) / n + (p) * np.log(n) / n
        if rs1 < rs2:
            linear_rs = np.sum(np.abs( edge2adj(linear_equalvar_obj['g'], p) - edges_star ))
            RS = np.vstack([
                RS,
                np.array([n, m, 'equal', 'linear', 
                    np.all( edge2adj(linear_equalvar_obj['g'], p) == edges_star),
                    linear_rs
                ])
            ])
        else:
            linear_rs = np.sum(np.abs( edge2adj(linear_diffvar_obj['g'], p) - edges_star ))
            RS = np.vstack([
                RS,
                np.array([n, m, 'equal', 'linear', 
                    np.all( edge2adj(linear_diffvar_obj['g'], p) == edges_star),
                    linear_rs
                ])
            ])



        # for unequal ======================================================================

        name = "./data/unequal_" + str(n) + "_" + str(m) + ".csv"
        X = np.loadtxt(name, delimiter=" ")
        n,p = X.shape
        basis_setting = {'type': 'spline', 'n_knots': 5, 'degree': 3, 'include_intercept': False}
        suff = gen_suff_stat(X, basis_setting)
        Sigma_hat = suff["Sigma"]
        R = suff["R"]


        # mip
        mips_equalvar_obj = Mips_DAG_equalvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs1 = loss_from_edges(mips_equalvar_obj['g'], X, basis_setting) + len(mips_equalvar_obj['g']) * np.log(n) / n
        mips_diffvar_obj = Mips_DAG_diffvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0)
        rs2 = loss_from_edges(mips_diffvar_obj['g'], X, basis_setting) + (p-1) * np.log(n) / n + len(mips_diffvar_obj['g']) * np.log(n) / n
        if rs1 < rs2:
            RS = np.vstack([
                RS, 
                np.array([n, m, 'unequal', 'Mips', 
                    np.all( edge2adj(mips_equalvar_obj['g'], p) == edges_star ),
                    np.sum(np.abs( edge2adj(mips_equalvar_obj['g'], p) - edges_star )) 
                ])
            ])
        else:
            RS = np.vstack([
                RS, 
                np.array([n, m, 'unequal', 'Mips', 
                    np.all( edge2adj(mips_diffvar_obj['g'], p) == edges_star ),
                    np.sum(np.abs( edge2adj(mips_diffvar_obj['g'], p) - edges_star )) 
                ])
            ])


        # linear
        Z = np.append(X, X, axis = 1)
        Sigma_hat = Z.T @ Z / n
        R = 1
        linear_equalvar_obj = Mips_DAG_equalvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0, time_limit=10)
        rs1 = loss_from_edges(linear_equalvar_obj['g'], X, basis_setting = {'type': 'linear'}) + len(linear_equalvar_obj['g']) * np.log(n) / n
        linear_diffvar_obj = Mips_DAG_diffvar(Sigma_hat, n, p, lam, R, moral, tau = 0, verbose = 0, time_limit=10)
        rs2 = loss_from_edges(linear_diffvar_obj['g'], X, basis_setting = {'type': 'linear'}) + len(linear_diffvar_obj['g']) * np.log(n) / n + (p) * np.log(n) / n
        if rs1 < rs2:
            linear_rs = np.sum(np.abs( edge2adj(linear_equalvar_obj['g'], p) - edges_star ))
            RS = np.vstack([
                RS,
                np.array([n, m, 'unequal', 'linear', 
                    np.all( edge2adj(linear_equalvar_obj['g'], p) == edges_star ),
                    linear_rs
                ])
            ])
        else:
            linear_rs = np.sum(np.abs( edge2adj(linear_diffvar_obj['g'], p) - edges_star ))
            RS = np.vstack([
                RS,
                np.array([n, m, 'unequal', 'linear', 
                    np.all( edge2adj(linear_diffvar_obj['g'], p) == edges_star ),
                    linear_rs
                ])
            ])
        

        print("finished", "m=", m, "n=", n)

In [ ]:
RS_df = pd.DataFrame(RS)
RS_df['new'] = 'exp(x)'
RS_df.to_csv("./results/exp(x)_python.csv", index=False) 